In [1]:
from deepeval.synthesizer import Synthesizer 
from deepeval.synthesizer.config import ContextConstructionConfig,StylingConfig

import os
from dotenv import load_dotenv

load_dotenv()

# Initialize the synthesizer with custom LLM and embedding models
def synthesize():
    """
    This function initializes the synthesizer 
    and generates golden outputs from documents.
    """
    styling_config=StylingConfig(
            input_format="Questions about homewoner insurance policy. What cover/not cover depending on home damage situation",
            expected_output_format="Detail answer on what covered/not covered for that accident based on policy",
            task="RAG chatbot for homewner insurance policy documents",
            scenario="Customer asking queries about their policy information"

        )
    
    synthesizer = Synthesizer(cost_tracking=True,styling_config=styling_config)

    goldens = synthesizer.generate_goldens_from_docs(
        document_paths=["../../evaluation/data/docs/sample_policy_doc_AU1234.pdf"],
        include_expected_output=True,
        max_goldens_per_context=2,  # Limit to one golden per context
        context_construction_config=ContextConstructionConfig(
            max_contexts_per_document=1,  # Limit to one context per document 
        )     
    )

    return synthesizer, goldens

synthesizer,goldens = synthesize()


c:\Users\sunny\Desktop\temp_env\env2\venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db

[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db

[Confident AI Synthesizer Log] SUCCESS: Context Construction: Utilizing 3 out of 58 chunks.

In [8]:
df = synthesizer.to_pandas()


In [11]:
df.to_excel('dataset.xlsx')

In [2]:
goldens

[Golden(input='What steps should I take after filing a homeowner insurance claim to ensure everything is processed smoothly?', actual_output=None, expected_output='Detail answer on what covered/not covered for that accident based on policy, and make sure it is concise and straight to the point, using supporting information in context.\n\nAfter filing a homeowner insurance claim, you should provide original purchase receipts or relevant documentation to prove your claim, obtain estimates for repairs, and assist with any enquiries regarding your claim. Do not admit responsibility, make agreements if held liable by others, abandon your property, or dispose of damaged items as we may need to see them. We may choose to repair, reinstate, replace, or pay you for loss/damage but not beyond the sum insured. Excess will be deducted from the claim. Report theft or damages due to theft to Police and obtain a reference number. If required, allow us to access your property for assessment and salvag

In [3]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import Golden
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

In [4]:
import sys
sys.path.append(os.path.abspath("../../"))

In [5]:
from evaluation.rag import RagPipeline
rag_obj = RagPipeline()
test_cases = []
for golden in goldens:
    res,text_chunks = rag_obj.execute_rag(golden.input)
    test_case = LLMTestCase(
        input=golden.input,
        actual_output=res,
        retrieval_context=text_chunks,
        expected_output=golden.expected_output,
    )
    test_cases.append(test_case)

c:\Users\sunny\Desktop\temp_env\env2\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
test_cases

[LLMTestCase(input='What steps should I take after filing a homeowner insurance claim to ensure everything is processed smoothly?', actual_output="After filing a homeowner insurance claim, there are several steps you can take to ensure the process goes smoothly:\n\n1. **Document Everything**: Keep a detailed record of all correspondence with your insurance company, including emails, letters, and notes from phone conversations. Document any expenses incurred as a result of the incident.\n\n2. **Meet Deadlines**: Adhere to any deadlines provided by your insurance company for submitting documents or additional information. This could include estimates for repair or restoration work.\n\n3. **Provide Accurate Information**: Make sure all information you provide is accurate and complete to avoid any delays in processing your claim.\n\n4. **Take Notes During Inspections**: If an insurance adjuster comes to inspect the damage, take notes during their visit. Ask for their contact information to